In [1]:
import math
import matplotlib.pyplot as plt #before plotting
import numpy as np
import pandas as pd #wes mckinney creation from R dataframes..2000 pages!
from pylab import *

from operator import attrgetter  #helps to split bins for strikes'distances

import blpapi
import datetime as dt
import tia.bbg.datamgr as dm

from tia.bbg import LocalTerminal 

#from VolSurface import VolSurface
from FXOption_v2 import FXOption
#from ExpReport_v2 import ExpReport

from datetime import date

#from datetime import datetime
from dateutil.relativedelta import relativedelta

from pandas.tseries.offsets import BDay #use last BD to bypass feed issues on hols and early mrng

pd.set_option('display.max_rows', 200) #https://stackoverflow.com/questions/16424493/pandas-setting-no-of-max-rows
pd.options.display.max_rows 
#pd.set_option('display.max_columns', 60) #https://stackoverflow.com/questions/16424493/pandas-setting-no-of-max-rows
pd.options.display.max_columns = None

In [3]:
?FXOption

In [2]:
start = dt.datetime.today() - BDay(1)
end = start

def get_market_data(pair, fields, start_date, end_date, period):
    '''imports historical data from bbg api, converts to dataframe '''
    resp = LocalTerminal.get_historical(pair, fields, start_date, end_date, period)
    prices = resp.as_frame()
    #use below to start at second row , to get rid of the extra column index ['XYZ Curncy'] title on top
    prices.columns = prices.columns.get_level_values(1) #layer one
    return prices


FVU2 = get_market_data('FVU2 Comdty', 'PX_LAST', start, end, 'DAILY')
FVZ2 = get_market_data('FVZ2 Comdty', 'PX_LAST', start, end, 'DAILY')
ESU2 = get_market_data('ESU2 Index', 'PX_LAST', start, end, 'DAILY')
ESZ2 = get_market_data('ESZ2 Index', 'PX_LAST', start, end, 'DAILY')


In [4]:
prices_map = {'FVU2': FVU2.iloc[0], 
              'FVZ2' : FVZ2.iloc[0],
             'ESU2': ESU2.iloc[0], 
             'ESZ2': ESZ2.iloc[0]}
#later version voesl back out from prices? curerntly FVA1 Comdty OVDV
def spot_assigner(row, dictio, spot_vol):

    #print(row)
    Inst = row['Inst'] #dicitonary key
    
    try:
        if spot_vol == "spot":
            spot = dictio[Inst]['PX_LAST']#.values
            #spot = dictio[Inst][0]
            #print(spot)
            return spot
        else:
            vol = dictio[Inst][1]
            return vol
    except:
        return np.nan
    
prices_map['ESU2']['PX_LAST']

4133.5

In [81]:
#inputs: 'Inst', 'Expiry', 'Call/Put', 'Strike', 'Size', 'Vol''

port = {0: ['ESU2', date(2022,9,16), 'Put',  4000, 20, 21],
        1: ['ESU2', date(2022,9,16), 'Put', 3500, -27, 29],
       }



df_port= pd.DataFrame(port).transpose()
df_port.columns =  ['Inst', 'Expiry', 'Call/Put', 'Strike', 'Size', 'Vol']
df_port = df_port.set_index(['Expiry'])
df_port.index = pd.to_datetime(df_port.index, dayfirst = True)
today  = pd.datetime.today()
df_port['Days'] = (df_port.index - today).days


def multiplier(Inst):
    ''' Eminis multpier 50, bonds 1000, stonks 100 usually '''
    
    if 'ES' in Inst:
        return 50
    elif 'FV' in Inst:
        return 1000
    elif 'SPY' in Inst:
        return 100
    
def distance(CallPutFlag, Spot, Strike):
    
    if CallPutFlag == 'Call':
        return np.log(Spot/Strike)*100
    return np.log(Strike/Spot)*100

df_port['Spot'] = df_port.apply(spot_assigner, dictio = prices_map, spot_vol = "spot",  axis = 1)

df_port['Distance%'] = df_port.apply(lambda x: distance(x['Call/Put'], x['Spot'], x['Strike']), axis = 1).round(1)

df_port['Multiplier'] = df_port['Inst'].apply(multiplier)
                                    
df_port['Price'] = df_port.apply(lambda x: FXOption(CallPutFlag = x['Call/Put'],
                                                   K = x['Strike'],
                                                   dateEnd = x['Days'],
                                                   notional = 1 ,# x['Size']*x['Strike'],
                                                   premConvention = 'DomesticPips').price(S = x['Spot'],
                                                                             sigma = x['Vol']/100,
                                                                             rFor = 0.02,
                                                                             rDom = 0.025), axis = 1).apply(lambda x: round(x,3))

df_port['Price$'] = df_port['Price'] * df_port['Multiplier']*df_port['Size']
#set notional = 1 to see it in % 0 to 100, get sense of moneyness
df_port['Delta'] = df_port.apply(lambda x: FXOption(CallPutFlag = x['Call/Put'],
                                                   K = x['Strike'],
                                                   dateEnd = x['Days'],
                                                   notional = 1,
                                                   premConvention = '').delta(S = x['Spot'],
                                                                              sigma = x['Vol']/100,
                                                                              rFor = 0.02, 
                                                                              rDom = 0.025, 
                                                                              convention = 'SEF'), axis = 1)
df_port['Delta$'] = (df_port['Delta']*df_port['Size']*df_port['Multiplier']*df_port['Strike']).apply(lambda x: round(x,1))

df_port

0.2586939615115944
0.09061664316493918
0.017867071471134855
0.28699178058300395
0.20061345354006463


C:\Users\miste\anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


,Inst,Call/Put,Strike,Size,Vol,Days,Spot,Distance%,Multiplier,Price,Price$,Delta,Delta$
Expiry,,,,,,,,,,,,,
2022-09-16,ESU2,Put,4000,5,21,47,4133.500000,-3.3,50,65.838,16459.5,-0.314134,-314133.6
2022-09-16,ESU2,Put,3500,-7,29,47,4133.500000,-16.6,50,9.060,-3171.0,-0.048648,59593.7
2022-12-16,ESZ2,Put,3500,10,28,138,4150.500000,-17.0,50,53.968,26984.0,-0.137435,-240512.1
2022-12-16,ESZ2,Put,3000,-10,34.5,138,4150.500000,-32.5,50,19.802,-9901.0,-0.049586,74379.6
2022-12-16,ESZ2,Call,4500,-10,18.4,138,4150.500000,-8.1,50,69.455,-34727.5,0.258694,-582061.4
2022-08-23,FVU2,Put,112.5,1,5.5,23,113.726562,-1.1,1000,0.185,185.0,-0.207235,-23313.9
2022-08-23,FVU2,Call,115.5,-4,4.5,23,113.726562,-1.5,1000,0.054,-216.0,0.090617,-41864.9
2022-08-23,FVU2,Call,116.5,4,4.5,23,113.726562,-2.4,1000,0.008,32.0,0.017867,8326.1
2022-11-23,FVZ2,Put,112.25,15,5.5,115,113.789062,-1.4,1000,0.694,10410.0,-0.303956,-511785.1


In [82]:
es = df_port[df_port['Inst'].str.contains('ES')]
print(es.loc[:,'Delta$'].sum())
es

-1002733.8


,Inst,Call/Put,Strike,Size,Vol,Days,Spot,Distance%,Multiplier,Price,Price$,Delta,Delta$
Expiry,,,,,,,,,,,,,
2022-09-16,ESU2,Put,4000,5,21,47,4133.5,-3.3,50,65.838,16459.5,-0.314134,-314133.6
2022-09-16,ESU2,Put,3500,-7,29,47,4133.5,-16.6,50,9.060,-3171.0,-0.048648,59593.7
2022-12-16,ESZ2,Put,3500,10,28,138,4150.5,-17.0,50,53.968,26984.0,-0.137435,-240512.1
2022-12-16,ESZ2,Put,3000,-10,34.5,138,4150.5,-32.5,50,19.802,-9901.0,-0.049586,74379.6
2022-12-16,ESZ2,Call,4500,-10,18.4,138,4150.5,-8.1,50,69.455,-34727.5,0.258694,-582061.4


In [83]:
fv = df_port[df_port['Inst'].str.contains('FV')]
print(fv.loc[:,'Delta$'].sum())
fv

-715926.9


,Inst,Call/Put,Strike,Size,Vol,Days,Spot,Distance%,Multiplier,Price,Price$,Delta,Delta$
Expiry,,,,,,,,,,,,,
2022-08-23,FVU2,Put,112.5,1,5.5,23,113.726562,-1.1,1000,0.185,185.0,-0.207235,-23313.9
2022-08-23,FVU2,Call,115.5,-4,4.5,23,113.726562,-1.5,1000,0.054,-216.0,0.090617,-41864.9
2022-08-23,FVU2,Call,116.5,4,4.5,23,113.726562,-2.4,1000,0.008,32.0,0.017867,8326.1
2022-11-23,FVZ2,Put,112.25,15,5.5,115,113.789062,-1.4,1000,0.694,10410.0,-0.303956,-511785.1
2022-11-23,FVZ2,Call,116,-15,5.5,115,113.789062,-1.9,1000,0.621,-9315.0,0.286992,-499365.7
2022-11-23,FVZ2,Call,117,15,5.5,115,113.789062,-2.8,1000,0.389,5835.0,0.200613,352076.6


In [7]:
?FXOption

In [59]:
opt_put = FXOption(CallPutFlag = 'Put', K = 112.5, dateEnd = date(2022,8,26), 
                notional = 1, 
                premConvention = "DomesticPips")

S = 114'113+23.25/32
sigma  = 0.055
rFor = 0.00
rDom = 0.02

print('value: '+'{:,}'.format(opt_put.price(S , sigma , rFor, rDom)))
print('delta: '+'{:,}'.format(opt_put.delta( S  , sigma , rFor, rDom, convention = 'SIF' )))
print('gamma: '+'{:,}'.format(opt_put.gamma(S, sigma , rFor, rDom, convention = 'SEF' )))

value: 0.14528855940325514
delta: -0.1619158540115863
gamma: 0.1630805047443507


In [26]:
0.0147*50

0.735

## 